<a href="https://colab.research.google.com/github/minhhieu9800/MLE501/blob/main/Assignment_MLE501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Name: Nguyễn Minh Hiếu </br>
Main Class: MSE#11HCM </br>
Student ID: 22MSE23062 </br>
Email: hieu22mse23062@fsb.edu.vn

# Assignment

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tabulate import tabulate

from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split

Load data

In [ ]:
# https://www.kaggle.com/datasets/mathchi/diabetes-data-set
url = 'https://minhhieu9800.github.io/MLE501/diabetes.csv'
df = pd.read_csv(url)

In [ ]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)

Build model

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 6s 7ms/step - loss: 3.9320 - accuracy: 0.5112 - val_loss: 2.2983 - val_accuracy: 0.5854
Epoch 2/100
62/62 [==============================] - 0s 4ms/step - loss: 2.8068 - accuracy: 0.5458 - val_loss: 1.8198 - val_accuracy: 0.5691
Epoch 3/100
62/62 [=====================

In [ ]:
model.save("model.h5")

In [ ]:
model = load_model("model.h5")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 4ms/step - loss: 0.6000 - accuracy: 0.7078
Loss:  0.5999835729598999
Accuracy:  0.7077922224998474


## Improving model by Hyperparameter Tuning (Keras Tuner)

In [ ]:
!pip install keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.6 MB/s eta 0:00:00


In [ ]:
# Create function: build Hyper model
def build_model(hp): 
  model = keras.Sequential()

  X = hp.Int('units_input', min_value=8, max_value=64, step=8)
  model.add(Dense(units=X, input_dim=8, activation='relu'))

  for i in range(hp.Int('num_layers', 1, 4)):
    Y = hp.Int('units_' + str(i), min_value=8, max_value=64, step=8)
    model.add(Dense(units=Y, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy'])
  
  return model

In [ ]:
# Create Tuner
tuner = kt.Hyperband (build_model, objective="val_accuracy", directory="tuner_dir_1", project_name="MLE501_Tuner_1")

In [ ]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 254 Complete [00h 00m 21s]
val_accuracy: 0.707317054271698

Best val_accuracy So Far: 0.8130081295967102
Total elapsed time: 00h 12m 55s


In [ ]:
# Get best model
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 24)                216       
                                                                 
 dense_2 (Dense)             (None, 24)                600       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 1s 8ms/step - loss: 0.6414 - accuracy: 0.7067 - val_loss: 0.5256 - val_accuracy: 0.7317
Epoch 2/100
62/62 [==============================

In [ ]:
best_model.save("best_model_hp.h5")

In [ ]:
modelOriginal = load_model("model.h5")
modelHp = load_model("best_model_hp.h5")

lossOriginal, accuracyOriginal = modelOriginal.evaluate(X_test, y_test)
lossHp, accuracyHp = modelHp.evaluate(X_test, y_test)

# create the DataFrame
df = pd.DataFrame({'Model': ['Original','Hyperparameter Tuning (Keras Tuner)'],
                   'Loss': [lossOriginal, lossHp],
                   'Accuracy': [accuracyOriginal, accuracyHp]})

# print the table using tabulate
print(tabulate(df, headers='keys', tablefmt='pipe', floatfmt='.4f'))

5/5 [==============================] - 0s 4ms/step - loss: 0.5180 - accuracy: 0.7403
|    | Model                               |   Loss |   Accuracy |
|---:|:------------------------------------|-------:|-----------:|
|  0 | Original                            | 0.6000 |     0.7078 |
|  1 | Hyperparameter Tuning (Keras Tuner) | 0.5180 |     0.7403 |


### Compared to the original model, the model improved by Hyperparameter Tuning has shown an improvement in accuracy. The change is generated from optimizing and selecting the Hyperparameters.




In [ ]:
# Initially, the model only had 3 layers with the following parameters:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The optimized model has layers with the following parameters:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 24)                216       
                                                                 
 dense_2 (Dense)             (None, 24)                600       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


## Improving model by Regularization (L2 Regularization)

In [ ]:
from keras import regularizers

In [ ]:
#Apply a lambda coefficient of 0.01 to both the first two Dense layers of the neural network architecture.

model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100
62/62 [==============================] - 2s 7ms/step - loss: 3.2479 - accuracy: 0.4114 - val_loss: 2.4286 - val_accuracy: 0.4634
Epoch 2/100
62/62 [==============================] - 0s 6ms/step - loss: 1.7630 - accuracy: 0.4745 - val_loss: 1.6000 - val_accuracy: 0.4797
Epoch 3/100
62/62 [==============================] - 0s 6ms/step - loss: 1.3122 - accuracy: 0.5234 - val_loss: 1.1785 - val_accuracy: 0.4959
Epoch 4/100
62/62 [==============================] - 0s 7ms/step - loss: 1.0171 - accuracy: 0.5764 - val_loss: 0.9445 - val_accuracy: 0.5366
Epoch 5/100
62/62 [==============================] - 0s 7ms/step - loss: 0.8989 - accuracy: 0.5764 - val_loss: 0.8432 - val_accuracy: 0.6423
Epoch 6/100
62/62 [==============================] - 0s 7ms/step - loss: 0.8604 - accuracy: 0.6293 - val_loss: 0.8163 - val_accuracy: 0.6911
Epoch 7/100
62/62 [==============================] - 0s 7ms/step - loss: 0.8417 - accuracy: 0.6456 - val_loss: 0.7931 - val_accuracy: 0.6748
Epoch 8/100
6

In [ ]:
model.save("best_model_regularization.h5")

In [ ]:
modelOriginal = load_model("model.h5")
modelRe = load_model("best_model_regularization.h5")

lossOriginal, accuracyOriginal = modelOriginal.evaluate(X_test, y_test)
lossRe, accuracyRe = modelRe.evaluate(X_test, y_test)

# create the DataFrame
df = pd.DataFrame({'Model': ['Original','Regularization (L2 Regularization)'],
                   'Loss': [lossOriginal, lossRe],
                   'Accuracy': [accuracyOriginal, accuracyRe]})

# print the table using tabulate
print(tabulate(df, headers='keys', tablefmt='pipe', floatfmt='.4f'))

5/5 [==============================] - 0s 4ms/step - loss: 0.6274 - accuracy: 0.7013
|    | Model                              |   Loss |   Accuracy |
|---:|:-----------------------------------|-------:|-----------:|
|  0 | Original                           | 0.6000 |     0.7078 |
|  1 | Regularization (L2 Regularization) | 0.6274 |     0.7013 |


### Compared to the original model, the model improved by Regularization, specifically using L2 Regularization, has shown a change in accuracy. The change is generated from adding a squared component of the weights to the loss function. This will make the weights smaller, resulting in a simpler model and avoiding overfitting.

## Improving model by Optimization (Adam -> RMSprop)

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100
62/62 [==============================] - 2s 11ms/step - loss: 8.0581 - accuracy: 0.3625 - val_loss: 3.2001 - val_accuracy: 0.3659
Epoch 2/100
62/62 [==============================] - 1s 8ms/step - loss: 1.6174 - accuracy: 0.5418 - val_loss: 1.0252 - val_accuracy: 0.6016
Epoch 3/100
62/62 [==============================] - 1s 9ms/step - loss: 1.1783 - accuracy: 0.5866 - val_loss: 0.9437 - val_accuracy: 0.5854
Epoch 4/100
62/62 [==============================] - 0s 8ms/step - loss: 1.0456 - accuracy: 0.5947 - val_loss: 0.8105 - val_accuracy: 0.6423
Epoch 5/100
62/62 [==============================] - 1s 15ms/step - loss: 0.9290 - accuracy: 0.5927 - val_loss: 0.7376 - val_accuracy: 0.6667
Epoch 6/100
62/62 [==============================] - 1s 19ms/step - loss: 0.8373 - accuracy: 0.5947 - val_loss: 0.7356 - val_accuracy: 0.5935
Epoch 7/100
62/62 [==============================] - 1s 10ms/step - loss: 0.7696 - accuracy: 0.5927 - val_loss: 0.6732 - val_accuracy: 0.5772
Epoch 8/1

In [ ]:
model.save("best_model_optimization.h5")

In [ ]:
modelOriginal = load_model("model.h5")
modelOp = load_model("best_model_optimization.h5")

lossOriginal, accuracyOriginal = modelOriginal.evaluate(X_test, y_test)
lossOp, accuracyOp = modelOp.evaluate(X_test, y_test)

# create the DataFrame
df = pd.DataFrame({'Model': ['Original','Optimization (Adam -> RMSprop)'],
                   'Loss': [lossOriginal, lossOp],
                   'Accuracy': [accuracyOriginal, accuracyOp]})

# print the table using tabulate
print(tabulate(df, headers='keys', tablefmt='pipe', floatfmt='.4f'))

5/5 [==============================] - 0s 4ms/step - loss: 0.6346 - accuracy: 0.7208
|    | Model                          |   Loss |   Accuracy |
|---:|:-------------------------------|-------:|-----------:|
|  0 | Original                       | 0.6000 |     0.7078 |
|  1 | Optimization (Adam -> RMSprop) | 0.6346 |     0.7208 |


### Compared to the original model, the model improved by Optimization has shown a change in accuracy, specifically by changing the optimizer from Adam to RMSprop. The optimization algorithm will change the way to calculate the gradients and update the weights during the training process. Specifically, these two algorithms use different methods to calculate gradients and update weights, so changing the optimizer will have an impact on the model's performance.

## Combining all three ways:

In [ ]:
# Create function build Hyper model
def build_best_model(hp): 
  model = keras.Sequential()

  X = hp.Int('units_input', min_value=8, max_value=64, step=8)
  model.add(Dense(units=X, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

  for i in range(hp.Int('num_layers', 1, 4)):
    Y = hp.Int('units_' + str(i), min_value=8, max_value=64, step=8)
    model.add(Dense(units=Y, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

  model.add(Dense(1, activation='sigmoid'))

  # Finding optimizer
  optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])

  # Finding learning_rate
  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  if optimizer == 'adam':
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])
  else:
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=lr), metrics=['accuracy'])
  
  return model

In [ ]:
# Create Tuner
tuner = kt.Hyperband (build_best_model, objective="val_accuracy", directory="tuner_dir_2", project_name="MLE501_Tuner_2")

In [ ]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 254 Complete [00h 00m 22s]
val_accuracy: 0.8211382031440735

Best val_accuracy So Far: 0.8292682766914368
Total elapsed time: 00h 13m 58s


In [ ]:
# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

print("units_input: ", best_hps.get('units_input'))
print("num_layers: ", best_hps.get('num_layers'))
print("optimizer: ", best_hps.get('optimizer'))
print("learning_rate: ", best_hps.get('learning_rate'))

units_input:  64
num_layers:  3
optimizer:  adam
learning_rate:  0.01


In [ ]:
# Get super model
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 24)                1560      
                                                                 
 dense_2 (Dense)             (None, 56)                1400      
                                                                 
 dense_3 (Dense)             (None, 64)                3648      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,249
Trainable params: 7,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 1

In [ ]:
best_model.save("best_model_super.h5")

In [ ]:
modelOriginal = load_model("model.h5")
modelHp = load_model("best_model_hp.h5")
modelRe = load_model("best_model_regularization.h5")
modelOp = load_model("best_model_optimization.h5")
modelSuper = load_model("best_model_super.h5")

lossOriginal, accuracyOriginal = modelOriginal.evaluate(X_test, y_test)
lossHp, accuracyHp = modelHp.evaluate(X_test, y_test)
lossRe, accuracyRe = modelRe.evaluate(X_test, y_test)
lossOp, accuracyOp = modelOp.evaluate(X_test, y_test)
lossSuper, accuracySuper = modelSuper.evaluate(X_test, y_test)

5/5 [==============================] - 0s 4ms/step - loss: 0.5382 - accuracy: 0.7208


In [ ]:
# create the DataFrame
df = pd.DataFrame({'Model': ['Original','Hyperparameter Tuning (Keras Tuner)','Regularization (L2 Regularization)','Optimization (Adam -> RMSprop)','Combined 3 ways'],
                   'Loss': [lossOriginal, lossHp, lossRe, lossOp, lossSuper],
                   'Accuracy': [accuracyOriginal, accuracyHp, accuracyRe, accuracyOp, accuracySuper]})

# print the table using tabulate
print(tabulate(df, headers='keys', tablefmt='pipe', floatfmt='.4f'))

|    | Model                               |   Loss |   Accuracy |
|---:|:------------------------------------|-------:|-----------:|
|  0 | Original                            | 0.6000 |     0.7078 |
|  1 | Hyperparameter Tuning (Keras Tuner) | 0.5180 |     0.7403 |
|  2 | Regularization (L2 Regularization)  | 0.6274 |     0.7013 |
|  3 | Optimization (Adam -> RMSprop)      | 0.6346 |     0.7208 |
|  4 | Combined 3 ways                     | 0.5382 |     0.7208 |


### Combining all three techniques has resulted in a better performing model with improved accuracy.